## Training eines neuronalen Netzes zur Erkennung von Ziffern

Bei Training der Snake-KI und beim Training des Chat-Bots haben wir neuronale Netze als "Black Box" verwendet. Heute wollen wir uns etwas genauer anschauen, wie das Training eines neuronalen Netzes funktioniert.

Wenn man eine neue Programmiersprache lernt, beginnt man meistens mit einem *Hello World* – einem kleinen Programm, das die Meldung `Hello World!` ausgibt.
Im Bereich der KI bzw. des maschinellen Lernens verwendet man gerne den *[MNIST Datensatz](https://de.wikipedia.org/wiki/MNIST-Datenbank)* als Beispiel verwendet.

In diesem Praktikum geht es um folgende Punkte:

**1. Wie trainiert man ein neuronales Netz?**<br>
   Wir lernen, wie man
   - Trainingsdaten lädt,
   - ein neuronales Netz aufbaut,
   - das neuronale Netz trainiert und
   - die Qualität des Netzes misst.


**2. Wovon hängt die Qualität der Erkennung ab?**<br>
   Jenachdem, 
   - wie wir unser Netz bauen, 
   - wie viele Trainingsdaten wir verwenden und 
   - wie lange wir das Netz trainieren
   sind die Ergebnisse besser oder schlechter. Dabei gibt es typische Probleme, die beim Training von KI-Modellen immer wieder auftauchen. Wie beim Fußball ist es 
   wichtig, als "Trainer" diese Probleme zu erkennen und zu wissen, was man dagegen tun kann.


In diesem Praktikum verwenden wir ein *Jupyter Notebook*. Darin kann man Erklärungen direkt in das Programm einbauen. Die folgende *Code-Zelle* gibt `Hello World!` aus, wenn Du sie ausführst.

In [ ]:
print("Hello World!")

Hier geht es mit dem Trainingsprogramm los. Die folgende Code-Zelle importiert die notwendigen Bibliotheken, die wir im folgenden benötigen.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

### Trainingsdaten laden

Jetzt laden wir die Trainingsdaten aus dem Internet und schauen uns ein paar Beispiele an:

In [ ]:
train_set = datasets.FashionMNIST(root="", train=True, transform=transforms.ToTensor(), download=True)
test_set = datasets.FashionMNIST(root="", train=False, transform=transforms.ToTensor(), download=True)

#### "Form" der Trainingsdaten

Wir schauen uns nun an, welche "Form" (*shape*) die Trainings- und Testdaten haben. Verstehst Du, was die drei Zahlen bedeuten?

In [ ]:
print("train_set shape: ", train_set.data.shape)
print("test_set shape: ", test_set.data.shape)

In [ ]:
train_loader = DataLoader(train_set, batch_size=64, pin_memory=True, num_workers=5, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, pin_memory=True, num_workers=5, shuffle=True)

#### Trainingsbeispiele ansehen

Die folgende Zelle lädt die ersten Trainingsdaten und zeigt ein paar Beispiele an. Damit können wir kontrollieren, dass alles geklappt hat.

In [ ]:
for images, labels in train_loader:
    # Wir laden die erste Portion Daten
    break;
    
figure = plt.figure()
num_images = 18
for i in range(num_images):
    plt.subplot(3, 6, i+1, title=f"{labels[i]}")
    plt.axis("off")
    plt.imshow(images[i].cpu().numpy().squeeze(), cmap="gray_r")

### Aufbau des neuronalen Netzes

Hier bauen wir nun das neuronale Netz. Wir versuchen es zunächst mit einem ganz einfachen Netz:

- Eine Eigabeschicht, die die $28 \cdot 28$ Pixel auf eine **versteckte Schicht** abbildet.
  Diese versteckte Schicht hat `hidden_size` Neuronen. Wir beginnen mit zwei Neuronen 
  (`hidden_size = 2`). 
- Die Eingabeschicht verwendet als **Aktivierungsfunktion** die 
  *[Rectified Linear Unit (ReLU)](https://de.wikipedia.org/wiki/Rectifier_(neuronale_Netzwerke))*
- Eine **Ausgabeschicht**, die für jede mögliche Ziffer ein Neuron enthält.
  Hier benötigen wir keine Aktivierungsfunktion – das am "stärksten feuernde" Neuron gewinnt am Ende
  bzw. die Werte werden automatisch so normiert, dass sie zwischen $0$ und $1$ liegen und in der Summe 
  $1$ ergeben ([Softmax](https://de.wikipedia.org/wiki/Softmax-Funktion)).
  
Zur Definition des Netzes benötigen wir eine Funktion `__init__()` um die Daten zu definieren, und eine 
Funktion `forward()` die Rechenschritte des Netzes ausführt.
Den ganzen mathematischen Rest erledigt die Bibliothek *PyTorch*.

In [ ]:
hidden_size = 128

class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        
        self.fc1 = nn.Linear(28*28, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 10)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = x.view(-1, 28*28)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out

In [ ]:
# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        # Convolutional layers
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        # Max pooling layer
        x = self.pool(x)
        # Flatten the tensor for fully connected layers
        x = x.view(-1, 64 * 12 * 12)
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # Output layer
        return F.log_softmax(x, dim=1)


In [ ]:
class AmelieCNN(nn.Module):
    def __init__(self):
        super(AmelieCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128*3*3, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.pool1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out = self.pool2(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out = self.relu3(out)
        out = self.pool3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        
        return out

### Neuronales Netz, Verlustfunktion und Optimierer anlegen

Neben dem neuronalen Netz erzeugen wir jetzt 
- Die Verlustfunktion, die wir optimieren. Sie misst, ob unser Netz die richtige Ziffer "vorschlägt" 
  und sich dabei möglichst sicher ist. Das heißt
  - die Ausgabe des Neurons der "richtigen" Ziffer soll möglichst hoch,
  - die Ausgabe der anderen Neuronen möglichst klein sein.
  Dafür verwendet man die sogenannte *[Kreuzentropie](https://de.wikipedia.org/wiki/Kreuzentropie)* 
  (engl. *Cross Entropy*).
- Den Optimierer, der beim Training die Verlustfunktion optimiert. Wir verwenden `optim.SGD`, wobei   
  *SGD* für *Stochastic Gradient Descent* steht. Anschaulich gesprochen verhält sich dieser Optimierer
  wie ein Skifahrer im Nebel – er fährt einfach dalang, wo es bergab geht, und hofft, dass er heil 
  im Tal ankommt.

In [ ]:
model = AmelieCNN()
model = model.cuda()

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

training_loss = []
testing_loss = []
training_acc = []
testing_acc = []

Hier schauen wir uns noch einmal an, wie viele Parameter unser Modell hat:

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(model)

### Training des Modells

Jetzt trainieren wir das Modell mit unseren Trainingsdaten.
Jeden Durchlauf über die kompletten Trainingsdaten nennt man *Epoche*. 

Da wir nur wenige Trainingsbilder verwenden, lassen wir unser Modell viele Epochen trainieren
(`epochs = 500`). 

Am Ende kannst Du die *Lernkurve* des Modells sehen. **Wenn sie noch nach oben gehst, kannst Du das Modell weiter trainieren, indem Du die Zelle noch einmal aufrufst.**


**Achtung:** Wenn Du die Zahl der Trainingsbilder erhöhst, solltest Du die Zahl der Epochen reduzieren, weil das Training sonst sehr lange dauert!

In [ ]:


epochs = 80

with tqdm(range(epochs)) as iterator:

    for epoch in iterator:
        train_loss = 0
        train_acc = 0

        model.train()
        for images, labels in train_loader:
            images, labels = images.cuda(), labels.cuda()
            output = model(images)
            optimizer.zero_grad()
            loss = loss_fn(output, labels)

            loss.backward()
            optimizer.step()

            prediction = torch.argmax(output, 1)
            train_acc += (prediction == labels).sum().item()
            train_loss += loss.item()

        training_acc.append(train_acc/len(train_set))
        training_loss.append(train_loss/len(train_set))

        test_loss = 0
        test_acc = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.cuda(), labels.cuda()
                output = model(images)
                loss = loss_fn(output, labels)
                prediction = torch.argmax(output, 1)
                test_acc += (prediction == labels).sum().item()
                test_loss += loss.item()        
            testing_acc.append(test_acc/len(test_set))
            testing_loss.append(test_loss/len(test_set))

        iterator.set_postfix_str(f"train_acc: {train_acc/len(train_set):.2f} test_acc: {test_acc/len(test_set):.2f} train_loss: {train_loss/len(train_set):.2f} test_loss: {test_loss/len(test_set):.2f}")

Die folgenden beiden Zellen geben die Lernkurven aus und zeigen die *Genauigkeit* und den *Verlust* jeweils für die Trainings- und Testdaten an.

**Was fällt Dir beim Vergleich der Kurven für Test- und Trainingsadten auf?**

In [ ]:
plt.plot(range(len(training_acc)), training_acc, label="train_acc")
plt.plot(range(len(training_acc)), testing_acc, label="test_acc")
plt.legend()
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.plot()

In [ ]:
plt.plot(range(epochs), training_loss, label="train_loss")
plt.plot(range(epochs), testing_loss, label="test_loss")
plt.legend()
plt.xlabel("epoch")
plt.ylabel("loss")
plt.plot()